# Generativní sítě

Rekurentní neuronové sítě (RNNs) a jejich varianty s bráněnými buňkami, jako jsou buňky Long Short Term Memory (LSTMs) a Gated Recurrent Units (GRUs), poskytují mechanismus pro modelování jazyka, tj. dokážou se naučit pořadí slov a poskytovat předpovědi pro další slovo v sekvenci. To nám umožňuje používat RNNs pro **generativní úlohy**, jako je běžná generace textu, strojový překlad nebo dokonce popisování obrázků.

V architektuře RNN, kterou jsme probírali v předchozí kapitole, každá jednotka RNN produkovala jako výstup další skrytý stav. Můžeme však také přidat další výstup ke každé rekurentní jednotce, což nám umožní generovat **sekvenci** (která má stejnou délku jako původní sekvence). Navíc můžeme použít jednotky RNN, které nepřijímají vstup na každém kroku, ale pouze berou nějaký počáteční stavový vektor a poté generují sekvenci výstupů.

V tomto notebooku se zaměříme na jednoduché generativní modely, které nám pomáhají generovat text. Pro zjednodušení vytvoříme **síť na úrovni znaků**, která generuje text písmeno po písmenu. Během trénování potřebujeme vzít nějaký textový korpus a rozdělit jej na sekvence znaků.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Vytváření slovníku znaků

Pro vytvoření generativní sítě na úrovni znaků je potřeba rozdělit text na jednotlivé znaky místo slov. Vrstva `TextVectorization`, kterou jsme používali dříve, to neumí, takže máme dvě možnosti:

* Ručně načíst text a provést tokenizaci "ručně", jak je uvedeno v [tomto oficiálním příkladu Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* Použít třídu `Tokenizer` pro tokenizaci na úrovni znaků.

Zvolíme druhou možnost. `Tokenizer` lze také použít k tokenizaci na úrovni slov, takže by mělo být snadné přepnout z tokenizace na úrovni znaků na tokenizaci na úrovni slov.

Pro tokenizaci na úrovni znaků je potřeba předat parametr `char_level=True`:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

Chceme také použít jeden speciální token k označení **konce sekvence**, který nazveme `<eos>`. Přidejme ho ručně do slovníku:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

Nyní, abychom zakódovali text do sekvencí čísel, můžeme použít:


In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Trénování generativní RNN pro vytváření nadpisů

Způsob, jakým budeme trénovat RNN na generování nadpisů zpráv, je následující. V každém kroku vezmeme jeden nadpis, který bude předán do RNN, a pro každý vstupní znak požádáme síť, aby vygenerovala následující výstupní znak:

![Obrázek ukazující příklad generování slova 'HELLO' pomocí RNN.](../../../../../translated_images/cs/rnn-generate.56c54afb52f9781d.webp)

Pro poslední znak naší sekvence požádáme síť, aby vygenerovala token `<eos>`.

Hlavní rozdíl mezi generativní RNN, kterou zde používáme, je ten, že budeme brát výstup z každého kroku RNN, a nejen z poslední buňky. Toho lze dosáhnout nastavením parametru `return_sequences` u buňky RNN.

Během trénování tedy bude vstupem do sítě sekvence zakódovaných znaků určité délky a výstupem bude sekvence stejné délky, ale posunutá o jeden prvek a ukončená tokenem `<eos>`. Minibatch bude obsahovat několik takových sekvencí, a bude nutné použít **padding**, aby se všechny sekvence zarovnaly.

Vytvořme funkce, které pro nás dataset transformují. Protože chceme sekvence zarovnávat na úrovni minibatch, nejprve dataset seskupíme pomocí `.batch()`, a poté použijeme `map` k provedení transformace. Transformační funkce tedy bude přijímat celý minibatch jako parametr:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Několik důležitých věcí, které zde děláme:
* Nejprve extrahujeme skutečný text z tensoru řetězců
* `text_to_sequences` převádí seznam řetězců na seznam tensorů s celými čísly
* `pad_sequences` doplňuje tyto tensory na jejich maximální délku
* Nakonec provedeme one-hot kódování všech znaků, a také posun a přidání `<eos>`. Brzy uvidíme, proč potřebujeme znaky kódované pomocí one-hot.

Nicméně, tato funkce je **Pythonická**, tj. nemůže být automaticky přeložena do výpočetního grafu Tensorflow. Pokud se pokusíme použít tuto funkci přímo ve funkci `Dataset.map`, dostaneme chyby. Musíme tento Pythonický volání uzavřít pomocí obálky `py_function`:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Note**: Rozlišování mezi Pythonovými a Tensorflow transformačními funkcemi může působit příliš složitě a možná se ptáte, proč dataset nepřetransformujeme pomocí standardních Python funkcí před jeho předáním do `fit`. I když to určitě lze udělat, použití `Dataset.map` má obrovskou výhodu, protože datová transformační pipeline je vykonávána pomocí Tensorflow výpočetního grafu, který využívá výpočty na GPU a minimalizuje potřebu přenosu dat mezi CPU/GPU.

Nyní můžeme vytvořit naši generátorovou síť a začít s trénováním. Může být založena na jakékoliv rekurentní buňce, kterou jsme probírali v předchozí jednotce (jednoduchá, LSTM nebo GRU). V našem příkladu použijeme LSTM.

Protože síť přijímá znaky jako vstup a velikost slovníku je poměrně malá, nepotřebujeme vrstvu pro vkládání (embedding layer), jednorázově zakódovaný vstup může přímo vstoupit do LSTM buňky. Výstupní vrstva bude `Dense` klasifikátor, který převede výstup LSTM na jednorázově zakódovaná čísla tokenů.

Navíc, protože pracujeme s sekvencemi proměnné délky, můžeme použít vrstvu `Masking`, která vytvoří masku ignorující vyplněné části řetězce. To není striktně nutné, protože nás příliš nezajímá vše, co přesahuje token `<eos>`, ale použijeme ji kvůli získání zkušeností s tímto typem vrstvy. `input_shape` bude `(None, vocab_size)`, kde `None` označuje sekvenci proměnné délky, a výstupní tvar je také `(None, vocab_size)`, jak můžete vidět z `summary`:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Generování výstupu

Nyní, když jsme model natrénovali, chceme jej použít k vytvoření nějakého výstupu. Nejprve potřebujeme způsob, jak dekódovat text reprezentovaný sekvencí čísel tokenů. K tomu bychom mohli použít funkci `tokenizer.sequences_to_texts`; nicméně tato funkce nefunguje dobře s tokenizací na úrovni znaků. Proto vezmeme slovník tokenů z tokenizeru (nazvaný `word_index`), vytvoříme reverzní mapu a napíšeme vlastní dekódovací funkci:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Nyní začneme s generováním. Začneme s nějakým řetězcem `start`, zakódujeme jej do sekvence `inp`, a poté při každém kroku zavoláme naši síť, aby určila další znak.

Výstup sítě `out` je vektor o `vocab_size` prvcích, který reprezentuje pravděpodobnosti jednotlivých tokenů. Nejpravděpodobnější číslo tokenu můžeme najít pomocí `argmax`. Tento znak pak připojíme k seznamu vygenerovaných tokenů a pokračujeme v generování. Tento proces generování jednoho znaku opakujeme `size` krát, abychom vygenerovali požadovaný počet znaků, a ukončíme jej předčasně, pokud narazíme na `eos_token`.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Ukázka výstupu během trénování

Protože nemáme žádné užitečné metriky, jako je *přesnost*, jediný způsob, jak můžeme zjistit, že se náš model zlepšuje, je **ukázka** generovaného řetězce během trénování. K tomu použijeme **callbacky**, tj. funkce, které můžeme předat funkci `fit` a které budou pravidelně volány během trénování.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


Tento příklad již generuje poměrně dobrý text, ale lze jej dále vylepšit několika způsoby:
* **Více textu**. Použili jsme pouze nadpisy pro náš úkol, ale můžete experimentovat s plným textem. Pamatujte, že RNN nejsou příliš dobré při práci s dlouhými sekvencemi, takže má smysl buď je rozdělit na kratší věty, nebo vždy trénovat na pevné délce sekvence s nějakou předem definovanou hodnotou `num_chars` (například 256). Můžete zkusit upravit výše uvedený příklad na takovou architekturu, přičemž se inspirujete [oficiálním tutoriálem Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/).
* **Vícevrstvé LSTM**. Má smysl vyzkoušet 2 nebo 3 vrstvy LSTM buněk. Jak jsme zmínili v předchozí části, každá vrstva LSTM extrahuje určité vzory z textu, a v případě generátoru na úrovni znaků můžeme očekávat, že nižší úroveň LSTM bude zodpovědná za extrakci slabik, zatímco vyšší úrovně - za slova a kombinace slov. To lze jednoduše implementovat předáním parametru počtu vrstev konstruktoru LSTM.
* Můžete také experimentovat s **GRU jednotkami** a zjistit, které fungují lépe, a s **různými velikostmi skrytých vrstev**. Příliš velká skrytá vrstva může vést k přeučení (např. síť se naučí přesný text), zatímco menší velikost nemusí produkovat dobré výsledky.


## Generování měkkého textu a teplota

V předchozí definici `generate` jsme vždy vybírali znak s nejvyšší pravděpodobností jako další znak v generovaném textu. To vedlo k tomu, že se text často "cyklil" mezi stejnými sekvencemi znaků znovu a znovu, jako v tomto příkladu:
```
today of the second the company and a second the company ...
```

Pokud se však podíváme na rozložení pravděpodobností pro další znak, může se stát, že rozdíl mezi několika nejvyššími pravděpodobnostmi není velký, např. jeden znak může mít pravděpodobnost 0,2, jiný 0,19 atd. Například při hledání dalšího znaku v sekvenci '*play*' může být další znak stejně dobře mezera nebo **e** (jako ve slově *player*).

To nás vede k závěru, že není vždy "spravedlivé" vybírat znak s vyšší pravděpodobností, protože výběr druhého nejvyššího může stále vést k smysluplnému textu. Je rozumnější **vzorkovat** znaky z rozložení pravděpodobností, které poskytuje výstup sítě.

Toto vzorkování lze provést pomocí funkce `np.multinomial`, která implementuje tzv. **multinomické rozložení**. Funkce, která implementuje toto **měkké** generování textu, je definována níže:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

Zavedli jsme další parametr nazvaný **teplota**, který slouží k určení, jak striktně bychom se měli držet nejvyšší pravděpodobnosti. Pokud je teplota 1.0, provádíme spravedlivé multinomiální vzorkování, a když teplota dosáhne nekonečna - všechny pravděpodobnosti se stanou rovnými a náhodně vybíráme další znak. Na níže uvedeném příkladu můžeme pozorovat, že text se stává nesmyslným, když příliš zvýšíme teplotu, a připomíná „cyklický“ tvrdě generovaný text, když se blíží k 0.



---

**Prohlášení**:  
Tento dokument byl přeložen pomocí služby pro automatický překlad [Co-op Translator](https://github.com/Azure/co-op-translator). I když se snažíme o přesnost, mějte prosím na paměti, že automatické překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho původním jazyce by měl být považován za autoritativní zdroj. Pro důležité informace doporučujeme profesionální lidský překlad. Neodpovídáme za žádná nedorozumění nebo nesprávné interpretace vyplývající z použití tohoto překladu.
